# 使用NTCIR給的relation資料集

In [53]:
!pip install transformers datasets accelerate


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
!nvidia-smi

Fri Apr 28 16:44:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.48       Driver Version: 517.48       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   34C    P8    22W / 200W |   7614MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 抓dataset

In [55]:
from datasets import list_datasets, load_dataset
from pprint import pprint

In [56]:

import json
with open("train_class.json","r")as f:
    pass
    dataset=json.load(f)
    print(type(dataset))

<class 'list'>


## dataset裡分成三個 train(訓練用) validation(每次訓縣完測試訓練情形用) 其中把train的520筆切出來當成test切出來當成test

In [57]:
print(dataset[5519])
print(dataset[0])
print(dataset[8])

['So, this $1 billion investment will help to enable $10 billion in cumulative Indian exports by 2025.', 0]
["I mean, sometimes it's not that you came up with some brilliant strategy, it's just like really good work consistently over a long period of time.", 0]
["On the GDPR changes, so we just started rolling out the GDPR controls in Europe and we're going to make all the same controls and settings available every way, which gives people the same opportunities to make the same choices.", 1]


In [58]:
train_data=dataset[0:7000]
test_data=dataset[7000:7753]
with open("dev_class.json","r")as f2:
    eval_data=json.load(f2)
#eval_data=dataset['validation']

In [59]:
import json
from pprint import pprint

pprint(dataset[5519])
pprint(dataset[0])
pprint(dataset[8])

['So, this $1 billion investment will help to enable $10 billion in cumulative '
 'Indian exports by 2025.',
 0]
["I mean, sometimes it's not that you came up with some brilliant strategy, "
 "it's just like really good work consistently over a long period of time.",
 0]
['On the GDPR changes, so we just started rolling out the GDPR controls in '
 "Europe and we're going to make all the same controls and settings available "
 'every way, which gives people the same opportunities to make the same '
 'choices.',
 1]


## 把每一筆資料集的每一項拆開

In [60]:
def read_data(dataset):
    #open file
    sentence1=[]
    sentence2=[]
    label=[]
    #idx=[]
    for text in dataset:
        sentence1.append(text[0])
        #sentence2.append(text[1])
        label.append(text[1])
    return sentence1,sentence2,label

In [61]:
"""def read_data(dataset):
    #open file
    sentence1=[]
    sentence2=[]
    label=[]
    #idx=[]
    for text in dataset:
        sentence1.append(text[0]+" [SEP] "+text[1])
        sentence2.append(text[1])
        label.append(text[2])
    return sentence1,sentence2,label"""

'def read_data(dataset):\n    #open file\n    sentence1=[]\n    sentence2=[]\n    label=[]\n    #idx=[]\n    for text in dataset:\n        sentence1.append(text[0]+" [SEP] "+text[1])\n        sentence2.append(text[1])\n        label.append(text[2])\n    return sentence1,sentence2,label'

In [62]:
train_sen1,train_sen2,trainlabel=read_data(train_data)
#trainlabel=train_data['label']
print(len(train_sen1))
print(train_sen1[0])

7000
I mean, sometimes it's not that you came up with some brilliant strategy, it's just like really good work consistently over a long period of time.


In [63]:
test_sen1,test_sen2,testlabel=read_data(test_data)
print(len(test_sen1))
print(test_sen1[0])

753
So really that's a big part of the focus.


In [64]:
eval_sen1,eval_sen2,evallabel=read_data(eval_data)
print(len(eval_sen1))
print(eval_sen1[0])

969
And in that context, of course, they're lifting and shifting some of the older workloads, but they're modernizing the entire business process flow.


In [65]:
print('train 資料集數量= ',len(train_data))
print('eval 資料集數量= ',len(eval_data))
print('test 資料集數量= ',len(test_data))

train 資料集數量=  7000
eval 資料集數量=  969
test 資料集數量=  753


In [66]:
print(trainlabel)

[0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 

# 選擇和使用tokenizer

In [67]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [68]:
from transformers import DebertaTokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

#from transformers import DebertaTokenizerFast

#tokenizer = DebertaTokenizerFast.from_pretrained("microsoft/deberta-base")

In [69]:
tokenizer("Hello world")["input_ids"]

[1, 31414, 232, 2]

In [70]:
tokenizer(" Hello world","hello world")["input_ids"]

[1, 20920, 232, 2, 42891, 232, 2]

In [71]:
tokenizer

PreTrainedTokenizerFast(name_or_path='microsoft/deberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [72]:
train_encodings = tokenizer(train_sen1, truncation=True, padding=True, return_tensors="pt")
print(train_encodings)
eval_encodings = tokenizer(eval_sen1, truncation=True, padding=True, return_tensors="pt")

{'input_ids': tensor([[   1,  100, 1266,  ...,    0,    0,    0],
        [   1, 8170,  150,  ...,    0,    0,    0],
        [   1, 2847,   13,  ...,    0,    0,    0],
        ...,
        [   1,  170,  348,  ...,    0,    0,    0],
        [   1,  170,  172,  ...,    0,    0,    0],
        [   1, 1121,   97,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [73]:
#把第二句改成label試試
#train_encodings = tokenizer(train_sen1, truncation=True, padding=True)
#print(train_encodings)
#eval_encodings = tokenizer(eval_sen1, truncation=True, padding=True)

In [74]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [75]:
print(tokenizer.decode(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0])
print("token_type_ids\n", train_encodings['token_type_ids'][0])
print("attention_mask\n", train_encodings['attention_mask'][0])

print(len(train_encodings['input_ids'][0]))
print(len(train_encodings['token_type_ids'][0]))

[CLS]I mean, sometimes it's not that you came up with some brilliant strategy, it's just like really good work consistently over a long period of time.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]
tensor([   1,  100, 1266,    6, 2128,   24,   18,   45,   14,   47,  376,   62,
          19,  103, 6967, 1860,    6,   24,   18,   95,  101,  269,  205,  173,
        6566,   81,   10,  251,  675,    9,   86,    4,    2,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        

## 加入label(答案)

In [76]:
def add_targets(encodings,label):
    encodings.update({'labels':label})
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)


In [77]:
print(train_encodings.keys())
print(tokenizer.decode(train_encodings['labels']))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
[PAD][CLS][PAD][CLS][CLS][CLS][PAD][PAD][CLS][CLS][CLS][CLS][PAD][PAD][PAD][PAD][PAD][PAD][PAD][CLS][PAD][PAD][PAD][CLS][PAD][CLS][PAD][CLS][PAD][PAD][PAD][CLS][PAD][PAD][PAD][CLS][PAD][PAD][CLS][PAD][PAD][CLS][CLS][PAD][CLS][CLS][PAD][CLS][CLS][CLS][PAD][CLS][CLS][CLS][CLS][PAD][PAD][CLS][PAD][PAD][PAD][PAD][PAD][CLS][PAD][PAD][CLS][CLS][CLS][PAD][CLS][PAD][CLS][PAD][CLS][PAD][PAD][CLS][PAD][PAD][PAD][PAD][CLS][CLS][CLS][CLS][CLS][CLS][PAD][CLS][PAD][PAD][CLS][CLS][CLS][PAD][CLS][CLS][PAD][CLS][CLS][CLS][PAD][PAD][CLS][PAD][PAD][CLS][CLS][PAD][PAD][PAD][PAD][CLS][CLS][CLS][PAD][PAD][CLS][PAD][CLS][PAD][CLS][PAD][PAD][PAD][PAD][CLS][PAD][PAD][CLS][PAD][PAD][CLS][PAD][CLS][PAD][PAD][PAD][PAD][PAD][CLS][PAD][CLS][CLS][CLS][CLS][PAD][CLS][CLS][CLS][PAD][PAD][PAD][PAD][PAD][CLS][CLS][PAD][CLS][CLS][CLS][CLS][CLS][CLS][PAD][PAD][CLS][PAD][PAD][PAD][CLS][PAD][PAD][PAD][PAD][CLS][CLS][CLS][PAD][PAD][PAD][PAD][CLS][PAD][PAD

In [78]:
len(eval_encodings['input_ids'])

969

# 定義dataset 並轉換成tensor格式


In [79]:
#eval[idx].clone().detach()
#torch.tensor(eval[idx])
from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [80]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [81]:
train_dataset[2]

C:\Users\lin\AppData\Local\Temp\ipykernel_30748\565021431.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}


{'input_ids': tensor([    1,  2847,    13,  1246,     6,    52,   393,  7849,    25,   203,
            11,    38,  1017,   486,    24,   786,    12, 28176,  1048,     6,
          4761,     8,   650,  1252,    19,    70,     9,     5, 10364, 16942,
            29,     4,     2,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [82]:
len(eval_dataset)

969

# 載入模型架構

In [83]:
from transformers import AutoTokenizer, DebertaForSequenceClassification
from transformers import AutoModel,DebertaModel
from transformers import AutoConfig

config = AutoConfig.from_pretrained("microsoft/deberta-base",)
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base",config=config) 
#model = DebertaModel.from_pretrained("microsoft/deberta-base")

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.d

In [84]:
print(model)

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

## 該來訓練模型囉

In [85]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 14     # 設定 training batch size 
eval_batch_size = 1      # 設定 eval batch size
num_train_epochs = 50      # 設定 epoch 

In [86]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [87]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

c:\Users\lin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


max_train_steps 25000


In [88]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

#eval_dataloader
metric = load_metric("accuracy")

真正開始訓練

In [89]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = './content/drive/Shareddrives'  # your folder


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    #print(batch)
    outputs = model(**batch)
    #loss = outputs
    loss = outputs.loss
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    #pred=outputs['labels']
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

04/28/2023 16:44:51 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no

04/28/2023 16:44:51 - INFO - __main__ - ***** Running training *****
04/28/2023 16:44:51 - INFO - __main__ -   Num examples = 7000
04/28/2023 16:44:51 - INFO - __main__ -   Num Epochs = 50
04/28/2023 16:44:51 - INFO - __main__ -   Instantaneous batch size per device = 14
04/28/2023 16:44:51 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 14
04/28/2023 16:44:51 - INFO - __main__ -   Gradient Accumulation steps = 1
04/28/2023 16:44:51 - INFO - __main__ -   Total optimization steps = 25000
Epoch:   0%|          | 0/50 [00:00<?, ?it/s]C:\Users\lin\AppData\Local\Temp\ipykernel_30748\565021431.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor

{'epoch': 0, 'step': 0, 'loss': 0.694381833076477}


{'epoch': 0, 'step': 50, 'loss': 0.6975188851356506}


{'epoch': 0, 'step': 100, 'loss': 0.5966178774833679}


{'epoch': 0, 'step': 150, 'loss': 0.6974810361862183}


...

{'epoch': 49, 'step': 300, 'loss': 9.383411452290602e-06}


{'epoch': 49, 'step': 350, 'loss': 1.032001728162868e-05}


{'epoch': 49, 'step': 400, 'loss': 5.211134975979803e-06}


{'epoch': 49, 'step': 450, 'loss': 6.241436039999826e-06}


Iteration: 100%|█████████▉| 499/500 [01:57<00:00,  4.25it/s]
04/28/2023 18:44:08 - INFO - __main__ - ***** Running eval *****
eval Iteration: 100%|██████████| 969/969 [00:25<00:00, 38.27it/s]
04/28/2023 18:44:33 - INFO - __main__ - epoch 49: {'accuracy': 0.7368421052631579}
Epoch: 100%|██████████| 50/50 [1:59:42<00:00, 143.66s/it]


## 最好的某次訓練成果

In [105]:
print(best_epoch)

{'epoch': 11, 'acc': 0.7492260061919505}


# 訓練成果驗證

In [106]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm

In [92]:
#cd drive

In [93]:
#ls

In [107]:
config = AutoConfig.from_pretrained("./content/drive/Shareddrives/epoch_11/config.json")
model = DebertaForSequenceClassification.from_pretrained("./content/drive/Shareddrives/epoch_11/pytorch_model.bin", config = config).to(device) 

In [95]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification

In [108]:
def mrpc_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

In [110]:
def mrpc_model(model, sen1):
  input_encodings = tokenizer([sen1], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

### 可以拿來玩的地方

In [111]:
sen1="lisa goes to school everyday"
sen2="lisa everyday goes to school"
#sen1="lisa is a singer"
#sen2="lisa is not a singer"

predict = mrpc_model(model, sen1)
print("sentence= : ", sen1)
#print("sentence= : ", sen2)

print("predict_label : ", predict.item())
if predict.item():
  print("premise")
else:
  print("claim")


sentence= :  lisa goes to school everyday
predict_label :  0
claim


In [112]:
cnt=0
errorcnt=0
pp=0
pc=0
cc=0
cp=0
for i in range(len(test_data)):
  cnt+=1
  sen1=test_data[i][0]
  answer=test_data[i][1]
  predict=mrpc_model(model,sen1)
  if predict.item()!=answer:
    #print(answer)
    #print(predict.item())
    errorcnt+=1
  if predict.item()==0 and answer==0:
    pp+=1
  elif predict.item()==0 and answer==1:
    pc+=1
  elif predict.item()==1 and answer==0:
    cp+=1
  elif predict.item()==1 and answer==1:
    cc+=1
print(cnt)
print(errorcnt)

753
188


In [114]:
p_precision=pp/(pp+pc+1)
p_recall=pp/(pp+cp+1)
c_precision=cc/(cc+cp+1)
c_recall=cc/(cc+pc+1)
p_f1=(2*p_precision*p_recall)/(p_precision+p_recall+1)
c_f1=(2*c_precision*c_recall)/(c_precision+c_recall+1)

print("acc = ",1-(errorcnt/cnt))
print("premise precision = ",p_precision)
print("premise recall = ",p_recall)
print("claim precision = ",c_precision)
print("claim recall = ",c_recall)
print("premise f1 = ",p_f1)
print("claim f1 = ",c_f1)

acc =  0.750332005312085
premise precision =  0.746268656716418
premise recall =  0.7731958762886598
claim precision =  0.7507082152974505
claim recall =  0.7220708446866485
premise f1 =  0.4580432392817883
claim f1 =  0.4384253521918146


結果存進google drive

In [101]:
"""from google.colab import drive
drive.mount('/content/gdrive')"""

"from google.colab import drive\ndrive.mount('/content/gdrive')"

In [102]:
#cd gdrive/MyDrive/Colab Notebooks

In [103]:
#torch.save(model,"./test_model2")

In [104]:
import torch
from transformers import AutoTokenizer, DebertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=num_labels)

labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.d